In [471]:
from __future__ import print_function, division

from keras.datasets import mnist
(X_train, y_train), (_, _) = mnist.load_data()



In [472]:

print(X_train[0].shape)

(28, 28)


# Get receipt dataset

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence

df_col = ["sentence","brand_name","info","index","content","total","thank_you"]
y_col = ["brand_name","info","index","content","total","thank_you"]
train_df = pd.read_csv('../text_classification/31-07-vigroupped.csv',   encoding='utf-8')

seed = 120
np.random.seed(seed)
train_df = shuffle(train_df)
train_df.head()

X_train = train_df["sentence"].fillna("fillna").values
Y_train = train_df[['brand_name', 'info', 'index', 'content', 'total', 'thank_you']].values

tokenizer = Tokenizer()
texts = X_train

tokenizer.fit_on_texts(texts) 
Tokenizer_vocab_size = len(tokenizer.word_index) + 1

X_train_encoded_words = tokenizer.texts_to_sequences(X_train)

maxWordCount= 10
maxDictionary_size=Tokenizer_vocab_size
X_train_encoded_padded_words = sequence.pad_sequences(X_train_encoded_words, maxlen=maxWordCount)

Using TensorFlow backend.


In [4]:
X_train_encoded_padded_words.shape
# Y_train

(644, 10)

In [475]:
tokenizer.word_index['ngừ']

1195

In [5]:
# Convert Y_train
# range(0,len(y_col))[5]
y_train = []
for row in Y_train:
    for index,col in enumerate(range(0,len(y_col))):
        if row[col] == 1:
            y_train.append(index)
print(len(y_train))

644


In [6]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

def padded_sequence_to_text(int_arr):
    
    padded_sequence = int_arr.reshape((maxWordCount))
    padded_sequence = padded_sequence.tolist()
#     print(padded_sequence)
    started = False
    word_seq = []
    for word in padded_sequence:
        if started:
            word_seq.append(word)
        else:
            if word != 0:
                started = True
                word_seq.append(word)
    
    sentences = list(map(sequence_to_text, [word_seq]))
    if len(sentences)>0:
        my_texts = []
        for word in sentences[0]:
            if word:
                my_texts.append(word)
            
        return ' '.join(my_texts)
    return None

def convert_y(y):
    result = []
    for index, col in enumerate(y_col):
        if index == y:
            result.append(1)
        else:
            result.append(0)
#     print(result)
    return result

def reshape_x_train(X_train_encoded_padded_words, r, c):
    x_train = []
    for row in X_train_encoded_padded_words:
        aa = np.array(row)

        aa = np.reshape(aa,(r ,c))
#         print(aa)
        x_train.append(aa)
    return np.array(x_train)
    

# Gan 

In [16]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self, r,c, token_vocal_size):
        self.img_rows = r
        self.img_cols = c
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        self.token_vocal_size = token_vocal_size
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(self.token_vocal_size, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.token_vocal_size*2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.token_vocal_size*4))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(self.token_vocal_size*2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(self.token_vocal_size))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        csvfile = 'cgan2d.csv'
        c = 6
#         self.img_cols = c
        noise = np.random.normal(0, 1, (c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        gen_imgs = Tokenizer_vocab_size*gen_imgs
        
        
        int_arr = np.array(gen_imgs, dtype='int')
#         print(int_arr[0])
        
        
#         print(len(int_arr[0,:,:,0]))
#         fig, axs = plt.subplots(r, c)
        cnt = 0

        for j in range(c):
            sentence = padded_sequence_to_text(int_arr[cnt])
#             result = convert_y(sampled_labels[cnt])
            if len(sentence) <= 0:
                continue
            print(sentence)
            cnt += 1
            df = pd.read_csv(csvfile)# Loading a csv file with headers 
            data = {
                'sentence':sentence,
            }
#             for index, col in enumerate(y_col):
#                 data[col] = result[index]
            df = df.append(data, ignore_index=True)
            df.to_csv(csvfile, index = False,  encoding='utf-8')


In [18]:
r = 5
c = 2
x_train = reshape_x_train(X_train_encoded_padded_words, r, c)
gan = GAN(r,c,Tokenizer_vocab_size)
gan.train(x_train,epochs=401, batch_size=32, sample_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 2392)              26312     
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 2392)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 1196)              2862028   
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 1196)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 1197      
Total params: 2,889,537
Trainable params: 2,889,537
Non-trainable params: 0
_________________________________________________________________


92 [D loss: 0.513512, acc.: 64.06%] [G loss: 1.557312]
93 [D loss: 0.349081, acc.: 87.50%] [G loss: 1.924021]
94 [D loss: 0.507735, acc.: 76.56%] [G loss: 1.468247]
95 [D loss: 0.533893, acc.: 68.75%] [G loss: 1.510409]
96 [D loss: 0.459330, acc.: 78.12%] [G loss: 1.578167]
97 [D loss: 0.307929, acc.: 92.19%] [G loss: 1.561700]
98 [D loss: 0.349282, acc.: 87.50%] [G loss: 1.421267]
99 [D loss: 0.291824, acc.: 89.06%] [G loss: 1.836307]
100 [D loss: 0.434606, acc.: 75.00%] [G loss: 1.581208]
tôm ngừ ngừ ngừ vị ngàn
0
5 ngừ cuộn ngừ đạu ngừ
tin
ngừ ngừ ngừ ngừ ngừ ngừ
ngừ 0 ngừ
101 [D loss: 0.392048, acc.: 82.81%] [G loss: 1.737487]
102 [D loss: 0.429860, acc.: 78.12%] [G loss: 1.670451]
103 [D loss: 0.336022, acc.: 81.25%] [G loss: 1.883909]
104 [D loss: 0.584530, acc.: 78.12%] [G loss: 1.495946]
105 [D loss: 0.380465, acc.: 82.81%] [G loss: 1.517966]
106 [D loss: 0.332591, acc.: 82.81%] [G loss: 1.744921]
107 [D loss: 0.378664, acc.: 82.81%] [G loss: 1.810818]
108 [D loss: 0.360261, ac

235 [D loss: 0.361190, acc.: 82.81%] [G loss: 2.139299]
236 [D loss: 0.371368, acc.: 79.69%] [G loss: 1.890949]
237 [D loss: 0.442867, acc.: 73.44%] [G loss: 1.834191]
238 [D loss: 0.401794, acc.: 78.12%] [G loss: 1.841414]
239 [D loss: 0.370381, acc.: 78.12%] [G loss: 1.845249]
240 [D loss: 0.336451, acc.: 82.81%] [G loss: 1.859740]
241 [D loss: 0.337714, acc.: 82.81%] [G loss: 2.071570]
242 [D loss: 0.256608, acc.: 90.62%] [G loss: 2.242397]
243 [D loss: 0.415657, acc.: 73.44%] [G loss: 2.131674]
244 [D loss: 0.329112, acc.: 90.62%] [G loss: 1.749195]
245 [D loss: 0.342945, acc.: 85.94%] [G loss: 1.544173]
246 [D loss: 0.396079, acc.: 82.81%] [G loss: 1.763821]
247 [D loss: 0.372587, acc.: 79.69%] [G loss: 1.999385]
248 [D loss: 0.248116, acc.: 93.75%] [G loss: 1.978695]
249 [D loss: 0.349782, acc.: 84.38%] [G loss: 2.076462]
250 [D loss: 0.326590, acc.: 87.50%] [G loss: 2.288359]
ngừ 000 phiếu
251 [D loss: 0.235474, acc.: 93.75%] [G loss: 2.225498]
252 [D loss: 0.312176, acc.: 82.81

380 [D loss: 0.312346, acc.: 85.94%] [G loss: 2.266357]
381 [D loss: 0.364495, acc.: 79.69%] [G loss: 1.953483]
382 [D loss: 0.286255, acc.: 84.38%] [G loss: 2.192087]
383 [D loss: 0.247929, acc.: 90.62%] [G loss: 2.134801]
384 [D loss: 0.289771, acc.: 87.50%] [G loss: 2.013503]
385 [D loss: 0.235784, acc.: 93.75%] [G loss: 2.355342]
386 [D loss: 0.335690, acc.: 84.38%] [G loss: 2.297014]
387 [D loss: 0.220987, acc.: 92.19%] [G loss: 2.507144]
388 [D loss: 0.261318, acc.: 90.62%] [G loss: 2.674321]
389 [D loss: 0.305058, acc.: 85.94%] [G loss: 2.475689]
390 [D loss: 0.291231, acc.: 87.50%] [G loss: 2.332996]
391 [D loss: 0.310568, acc.: 90.62%] [G loss: 2.246202]
392 [D loss: 0.368567, acc.: 82.81%] [G loss: 2.174954]
393 [D loss: 0.286234, acc.: 87.50%] [G loss: 2.448154]
394 [D loss: 0.168018, acc.: 93.75%] [G loss: 2.435511]
395 [D loss: 0.347697, acc.: 79.69%] [G loss: 2.530761]
396 [D loss: 0.434484, acc.: 76.56%] [G loss: 2.252105]
397 [D loss: 0.289210, acc.: 87.50%] [G loss: 2.